In [3]:
import os
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
import glob

In [9]:
# Paths to TFRecord files
train_tfrecord_paths = [
    '/home/mostafabakr8962/Desktop/object_detection/tfrecord/train/Mobile-Phones.tfrecord',
    '/home/mostafabakr8962/Desktop/object_detection/tfrecord/train/People.tfrecord',
    '/home/mostafabakr8962/Desktop/object_detection/tfrecord/train/glasses.tfrecord'
]
val_tfrecord_paths = [
    '/home/mostafabakr8962/Desktop/object_detection/tfrecord/val/Mobile-Phones.tfrecord',
    '/home/mostafabakr8962/Desktop/object_detection/tfrecord/val/People.tfrecord',
    '/home/mostafabakr8962/Desktop/object_detection/tfrecord/val/glasses.tfrecord'
]

In [10]:
# Parsing function for TFRecord files including bounding boxes
def parse_tfrecord_fn(example):
    feature_description = {
        'image/encoded': tf.io.FixedLenFeature([], tf.string),
        'image/object/class/label': tf.io.VarLenFeature(tf.int64),
        'image/object/bbox/xmin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/xmax': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymax': tf.io.VarLenFeature(tf.float32),
    }
    
    # Parse the example
    example = tf.io.parse_single_example(example, feature_description)

    # Decode and preprocess the image
    image = tf.io.decode_jpeg(example['image/encoded'], channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32) / 255.0

    # Handle bounding boxes: If there are no bounding boxes, return default values
    xmin = tf.sparse.to_dense(example['image/object/bbox/xmin'], default_value=0.0)
    ymin = tf.sparse.to_dense(example['image/object/bbox/ymin'], default_value=0.0)
    xmax = tf.sparse.to_dense(example['image/object/bbox/xmax'], default_value=1.0)
    ymax = tf.sparse.to_dense(example['image/object/bbox/ymax'], default_value=1.0)
    
    # Stack bounding boxes
    bboxes = tf.stack([xmin, ymin, xmax, ymax], axis=-1)
    
    # Ensure bounding box has the right shape (num_boxes, 4)
    tf.debugging.assert_rank(bboxes, 2, message="Bounding boxes should have rank 2")

    # Extract and process the label
    label = tf.sparse.to_dense(example['image/object/class/label'], default_value=0)
    
    # Ensure that only one label is used for each image (e.g., choose the first if multiple labels exist)
    label = tf.reduce_max(label)  # Choose the highest label for simplicity
    
    # Convert labels to one-hot encoding
    num_classes = 4  # Set number of classes
    labels_one_hot = tf.one_hot(tf.cast(label, tf.int32), depth=num_classes)

    # Log shapes of bounding boxes and labels for debugging
    tf.print("Image shape:", tf.shape(image))
    tf.print("Bounding boxes shape:", tf.shape(bboxes))
    tf.print("Labels shape:", tf.shape(labels_one_hot))

    return image, labels_one_hot


In [75]:
def load_dataset(tfrecord_dir_paths, batch_size, repeat=False):
    # Get a list of all TFRecord files in the directories
    tfrecord_paths = []
    for dir_path in tfrecord_dir_paths:
        tfrecord_paths.extend(glob.glob(dir_path + '/*.tfrecord'))

    # Load the dataset
    dataset = tf.data.TFRecordDataset(tfrecord_paths)
    dataset = dataset.map(parse_tfrecord_fn, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    dataset = dataset.cache()  # Cache data after loading once
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)  # Prefetch to optimize performance

    if repeat:
        dataset = dataset.repeat()

    return dataset

# Load the datasets
train_dir_paths = ['/home/mostafabakr8962/Desktop/object_detection/tfrecord/train']
val_dir_paths = ['/home/mostafabakr8962/Desktop/object_detection/tfrecord/val']

batch_size = 16
train_dataset = load_dataset(train_dir_paths, batch_size, repeat=True)
val_dataset = load_dataset(val_dir_paths, batch_size, repeat=True)

In [76]:
num_classes = 4
print(f"Number of classes: {num_classes}")

# Build the model
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # Softmax for multi-class classification
])

Number of classes: 4


In [77]:
# Compile the model with categorical crossentropy loss
optimizer = Adam(learning_rate=1e-4)
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',  # Categorical crossentropy for multi-class classification
    metrics=['accuracy', 'AUC']  # Add AUC metric
)


In [1]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5 , restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

def get_steps_per_epoch(dataset):
    cardinality = tf.data.experimental.cardinality (dataset).numpy()
    return cardinality if cardinality > 0 else 100  # Fallback value

train_steps_per_epoch = get_steps_per_epoch(train_dataset)
val_steps_per_epoch = get_steps_per_epoch(val_dataset)

if train_steps_per_epoch > 0 and val_steps_per_epoch > 0:
    history = model.fit(
        train_dataset,
        epochs=50,
        validation_data=val_dataset,
        steps_per_epoch=train_steps_per_epoch,
        validation_steps=val_steps_per_epoch,
        callbacks=[early_stopping, reduce_lr],
    )
else:
    print("Error: One or both datasets are empty!")

NameError: name 'EarlyStopping' is not defined